<a href="https://colab.research.google.com/github/devd-09/Gee_Projects/blob/main/ML_bigdata_lulc_monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [174]:
import geemap.geemap as map
import ee
import os
ee.Authenticate()
ee.Initialize(project = 'debayan2002')
vis_map=map.Map()

In [175]:
wb_boundary = ee.FeatureCollection('projects/debayan2002/assets/wb_boundary')
vis_map.addLayer(wb_boundary,{},'West Bengal Boundary') #Importing West Bengal Shapefile
vis_map.centerObject(wb_boundary,zoom=7)

In [176]:
value_visual = {'min':10,'max':100,'palette':['#006400','#ffbb22','#ffff4c','#f096ff','#fa0000','#b4b4b4','#f0f0f0','#0064c8','#0096a0','	#00cf75','#fae6a0']}
Legend = {'Tree Cover':'#006400',
          'Shrubland':'#ffbb22',
          'Grassland':'#ffff4c',
          'Cropland':'#f096ff',
          'Built-up':'#fa0000',
          'Bare / sparse vegetation':'#b4b4b4',
          'Snow and ice':'#f0f0f0',
          'Permanent water bodies':'#0064c8',
          'Herbaceous wetland':'#0096a0',
          'Mangroves':'#00cf75',
          'Moss and lichen':'#fae6a0'} #adding legend and visualisation parameters for lulc as per ESA Worldcover

In [177]:
wb_esa = ee.ImageCollection('ESA/WorldCover/v200').mean().clip(wb_boundary).select('Map')
vis_map.addLayer(wb_esa ,value_visual, 'West Bengal ESA Worldcover');
vis_map.add_legend(title="My Legend", legend_dict=Legend, **value_visual) #adding base raster dataset for extraction

In [178]:
landsat8_Data = ee.ImageCollection("LANDSAT/LC08/C02/T1").filterDate('2024-01-01', '2024-12-31').filterMetadata('CLOUD_COVER','less_than',5).mean().clip(wb_boundary) #defining landsat data on which classification will be done

In [179]:
TP = wb_esa.sample(**{
    'region':wb_boundary.geometry(),
    'scale':20,
    'numPixels':4000,
    'seed':1,
    'geometries':True
    })
vis_map.addLayer(TP,{},'Training Points')
#Adding the training points
#adding 4000 training points

In [180]:
bands = ['B1','B2','B3','B4','B5','B6']
label = 'Map'
training = landsat8_Data.select(bands).sampleRegions(**{'collection':TP,
                                       'properties':[label],
                                       'scale':30})
trained = ee.Classifier.smileKNN().train(training,label,bands)
result = landsat8_Data.select(bands).classify(trained) #training ML model smileKNN

In [171]:
vis_map.addLayer(result,value_visual,'Supervised_classification')
vis_map.addLayerControl(position="bottomleft")
vis_map

Map(center=[23.791561107942837, 87.97928006427928], controls=(WidgetControl(options=['position', 'transparent_…

The code written below is to download the classified image directly into your pc. In this case it is not allowed to download files over 50.6 mb. As a result I had to set the scale to 250.

If you want high resolution image of your study area you can set the scale as per your need but save it directly to your google drive . code is written below -

task = ee.batch.Export.image.toDrive(
    image=result,
    description='LULC_WB',
    folder='GEE_Exports',
    fileNamePrefix='LULC_WB',
    scale=30,
    region=wb_boundary.geometry(),
    maxPixels=1e13  # Increase max allowed pixels
)
task.start()

In [172]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'LULC_WB.tif')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
map.ee_export_image(
    result,
    filename=filename,
    scale=250,  # Set a reasonable scale (e.g., 30m)
    region=wb_boundary.geometry(),  # Ensure the region is correct
    file_per_band=False
) #to download lulc tiff file

Generating URL ...
Please wait ...
Data downloaded to /root/Downloads/LULC_WB.tif


In [173]:
output_path=os.path.join(os.path.expanduser("~"), "Downloads")
out_lulc_stats = os.path.join(output_path, "wb_lulc_2024.csv")
if not os.path.exists(output_path):
    os.makedirs(output_path)
map.zonal_stats(result,wb_boundary, out_lulc_stats, stat_type="HIST", scale=250) #Downloading statistical data that have been used

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /root/Downloads/wb_lulc_2024.csv
